In [9]:
%load_ext autoreload
%autoreload 2

%autosave 120

import pandas as pd
import glob
import numpy as np
import os
from modules.utilities import are_bugs_from_tangled, has_bug, is_same_history, parse_hash_delimited_string, has_one_bug

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 120 seconds


In [10]:
csv_files = glob.glob("./data/Processed/*.csv")

# True Buggy Methods

In [11]:
true_buggy_methods = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    for index, row in df.iterrows():
        if has_bug(row["Buggycommiit"]) and is_same_history([row["RiskyCommit"], row["PotentiallyBuggycommit"],row["Buggycommiit"]]) and not are_bugs_from_tangled(row["TangledWMoveandFileRename"], row["Buggycommiit"]):
            true_buggy_methods.append({
                "Project": os.path.basename(csv_file).replace(".csv",""),
                "File": row["file"]
            })

In [12]:
true_buggy_methods = pd.DataFrame(true_buggy_methods)
true_buggy_methods.to_csv("./data/TrueBuggyMethods.csv", index=False)

# True Not-Buggy Methods

In [13]:
true_not_buggy_methods = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    for index, row in df.iterrows():
        if not has_bug(row["Buggycommiit"]) and is_same_history([row["RiskyCommit"], row["PotentiallyBuggycommit"],row["Buggycommiit"]]):
            true_not_buggy_methods.append({
                "Project": os.path.basename(csv_file).replace(".csv",""),
                "File": row["file"]
            })

In [14]:
true_not_buggy_methods = pd.DataFrame(true_not_buggy_methods)
true_not_buggy_methods.to_csv("./data/TrueNotBuggyMethods.csv", index=False)

# Tangled Buggy Methods

In [15]:
tangled_buggy_methods = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file,delimiter="\t")
    for index, row in df.iterrows():
        if has_one_bug(row["Buggycommiit"]) and is_same_history([row["RiskyCommit"], row["PotentiallyBuggycommit"],row["Buggycommiit"]]) and are_bugs_from_tangled(row["TangledWMoveandFileRename"], row["Buggycommiit"]):
            #print(row["file"], row["RiskyCommit"], row["TangledWMoveandFileRename"])
            change_list = parse_hash_delimited_string(row["TangledWMoveandFileRename"])
            change_list.reverse()
            history = parse_hash_delimited_string(row["Buggycommiit"])
            history.reverse()
            
            for index, (change_count, is_buggy) in enumerate(zip(change_list, history)):
                if is_buggy == 1:
                    tangled_buggy_methods.append({
                        "Project": os.path.basename(csv_file).replace(".csv",""),
                        "File": row["file"],
                        "HistoryIndex": index,
                        "FileChangeCount": change_count
                    })

In [16]:
tangled_buggy_methods = pd.DataFrame(tangled_buggy_methods)
tangled_buggy_methods = tangled_buggy_methods[tangled_buggy_methods["FileChangeCount"] > 1]
tangled_buggy_methods = tangled_buggy_methods[tangled_buggy_methods["FileChangeCount"] <=5]
tangled_buggy_methods.to_csv("./data/TangledBuggyMethods.csv", index=False)